In [25]:
import tensorflow
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [26]:
from tensorflow.keras.models import load_model
model = load_model('omr_cnn.keras')

In [27]:
img = cv2.imread('OMR1.jpg')

In [28]:
img1= cv2.resize(img,(2000, 2500))

In [29]:
left_half = img1[760:2290, 350:750]
right_half = img1[760:2290, 1130:1510]

In [30]:
h, w, c = left_half.shape
h1, w1, c1 = right_half.shape

In [31]:
img2 = cv2.cvtColor(left_half, cv2.COLOR_BGR2GRAY)
img2 = cv2.GaussianBlur(img2, (5, 5), 1)
img3 = cv2.cvtColor(right_half, cv2.COLOR_BGR2GRAY)
img3 = cv2.GaussianBlur(img3, (5, 5), 1)

In [33]:
images=[]
slice_height = h // 15
slice_width = w
for i in range(15):
  start = i * slice_height
  end = start + slice_height
  img_slice = img2[start:end, :]
  img_slice = cv2.resize(img_slice, (280, 56))
  img_slice = np.expand_dims(img_slice, axis=-1)
  img_slice =  img_slice/255
  images.append(img_slice)

In [34]:
slice_height = h1 // 15
slice_width = w1
for i in range(15):
  start = i * slice_height
  end = start + slice_height
  img_slice = img3[start:end, :]
  img_slice = cv2.resize(img_slice, (280, 56))
  img_slice = np.expand_dims(img_slice, axis=-1)
  img_slice =  img_slice/255
  images.append(img_slice)

In [35]:
images_array = np.array(images)
images_array = images_array.reshape(-1, 56, 280, 1)
pred = model.predict(images_array)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


In [36]:
threshold = 0.5
predicted_answers = []
for prediction in pred:
    multi_label = (prediction >= threshold).astype(int)
    predicted_answers.append(multi_label)

Reading the answer & returning the result 

In [37]:
df = pd.read_csv("answer_key.csv")

In [39]:
from sklearn.preprocessing import MultiLabelBinarizer

df["answer"] = df["answer"].apply(lambda x: x.split(","))

mlb = MultiLabelBinarizer(classes=["A","B","C","D","E"])
encoded = mlb.fit_transform(df["answer"])

ans = pd.DataFrame(encoded, columns=mlb.classes_)
ans = ans.to_numpy()


In [40]:
correct = 0
wrong = 0
for i in range(30):
    if (predicted_answers[i]== ans[i]).all():
        correct+=1
    else:
        wrong+=1
if correct>=20:
    print(f"Excelent, You scored {correct} marks.")
elif correct>=10 and correct<20:
    print(f"Bingo, You scored {correct} marks.")
else:
    print(f"You scored {correct} marks. Needs Improvements")


Excelent, You scored 24 marks.
